In [1]:
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
import os
import numpy as np
import pandas as pd
import pickle as p
import xgboost as xgb

In [2]:
app = Flask(__name__)

MODEL_FOLDER = './models'
FEATURE_FOLDER = './features'
app.config['MODEL_FOLDER'] = MODEL_FOLDER
app.config['FEATURE_FOLDER'] = FEATURE_FOLDER

feature_file_path = './features/feature.pkl'
model_file_path = './models/model_old.pkl' #test on old

In [3]:
import requests

In [4]:
feature_file_path

'./features/feature.pkl'

# load feature

In [5]:
threshold = 0.1
data = {"int_rate": 0.34, "loan_amnt": 15000, "term": 36, "fund_rate": 0.4, 
"bc_open_to_buy": 9.0, "total_il_high_credit_limit": 2.8, "dti": 1.31, "annual_inc": 66, "bc_util": 10000}
pd.read_pickle(open(feature_file_path, 'rb'))

with open(feature_file_path, 'rb') as f:
    feature = pd.read_pickle(f)

In [6]:
feature.T.sort_index()

,28007
acc_now_delinq,0.0
acc_open_past_24mths,7.0
addr_state_AK,0.0
addr_state_AL,0.0
addr_state_AR,0.0
...,...
total_cu_tl,NaN
total_il_high_credit_limit,68217.0
total_rev_hi_lim,24900.0
train_flg,1.0


In [7]:
feature.columns.sort_values()

Index(['acc_now_delinq', 'acc_open_past_24mths', 'addr_state_AK',
       'addr_state_AL', 'addr_state_AR', 'addr_state_AZ', 'addr_state_CA',
       'addr_state_CO', 'addr_state_CT', 'addr_state_DC',
       ...
       'tot_hi_cred_lim', 'total_acc', 'total_bal_ex_mort', 'total_bal_il',
       'total_bc_limit', 'total_cu_tl', 'total_il_high_credit_limit',
       'total_rev_hi_lim', 'train_flg', 'zip_code_freq'],
      dtype='object', length=180)

In [8]:
tmp = model.feature_names

NameError: name 'model' is not defined

In [ ]:
tmp.sort()
tmp

['accnowdelinq',
 'accopenpast24mths',
 'addrstate_freq',
 'allutil',
 'annualinc',
 'annualincjoint',
 'applicationtype_Individual',
 'applicationtype_Joint App',
 'avgcurbal',
 'bcopentobuy',
 'bcutil',
 'chargeoffwithin12mths',
 'collections12mthsexmed',
 'delinq2yrs',
 'delinqamnt',
 'dti',
 'dtijoint',
 'earliestcrline_month',
 'emplength',
 'emptitle_freq',
 'ficorangehigh',
 'ficorangelow',
 'grade',
 'homeownership_ANY',
 'homeownership_MORTGAGE',
 'homeownership_NONE',
 'homeownership_OWN',
 'homeownership_RENT',
 'ilutil',
 'initialliststatus_f',
 'initialliststatus_w',
 'inqfi',
 'inqlast12m',
 'inqlast6mths',
 'installment',
 'intrate',
 'loanamnt',
 'maxbalbc',
 'mortacc',
 'mosinoldilacct',
 'mosinoldrevtlop',
 'mosinrcntrevtlop',
 'mosinrcnttl',
 'mthssincelastdelinq',
 'mthssincelastmajorderog',
 'mthssincelastrecord',
 'mthssincercntil',
 'mthssincerecentbc',
 'mthssincerecentbcdlq',
 'mthssincerecentinq',
 'mthssincerecentrevoldelinq',
 'numacctsever120pd',
 'numactvb

# fix feature

In [9]:
# fix feature
feature['bc_open_to_buy'] = data['bc_open_to_buy']
feature['total_il_high_credit_limit'] = data['total_il_high_credit_limit']
feature['dti'] = data['dti']
feature['annual_inc'] = data['annual_inc']
feature['bc_util'] = data['bc_util']
feature['int_rate'] = data['int_rate']/100
# high wie
if data['term'] == 36:
    feature['term_ 36 months'] = 1
    feature['term_ 60 months'] = 0
else:
    feature['term_ 36 months'] = 0
    feature['term_ 60 months'] = 1
# funded_amnt
feature['loan_amnt'] = data['loan_amnt']
loan_rate = data['fund_rate']/100
feature['funded_amnt'] = loan_rate * feature['loan_amnt']

# load model

In [10]:
dtrain = xgb.DMatrix(feature, missing=np.NAN)
modelfile = model_file_path
with open(modelfile, 'rb') as f:
    model = p.load(f)

In [11]:
ff = feature.columns.to_list()
mf = model.feature_names
print(
    len(ff),
    len(mf)
)

180 180


In [12]:
set(mf).difference(ff)

set()

# model prediction

In [57]:
prediction = model.predict(dtrain)
prediction = float(prediction)

ValueError: feature_names mismatch: ['secappopenactil', 'bcopentobuy', 'numactvbctl', 'mthssincelastrecord', 'totcurbal', 'secappnumrevaccts', 'secappmthssincelastmajorderog', 'secappcollections12mthsexmed', 'mosinrcnttl', 'openrv12m', 'inqlast12m', 'mortacc', 'mthssincelastdelinq', 'numactvrevtl', 'tothicredlim', 'revolbaljoint', 'ficorangehigh', 'maxbalbc', 'openil24m', 'numbctl', 'totalrevhilim', 'totcollamt', 'secappopenacc', 'numtl120dpd2m', 'secappficorangelow', 'totalbalexmort', 'annualincjoint', 'ficorangelow', 'totalacc', 'mthssincerecentbcdlq', 'installment', 'numrevaccts', 'pubrec', 'accopenpast24mths', 'numoprevtl', 'avgcurbal', 'secappmortacc', 'openactil', 'dti', 'openacc6m', 'totalbalil', 'delinq2yrs', 'mosinoldilacct', 'annualinc', 'inqfi', 'numtl30dpd', 'loanamnt', 'numsats', 'mthssincerecentrevoldelinq', 'mthssincerecentbc', 'secappchargeoffwithin12mths', 'revolbal', 'inqlast6mths', 'mthssincercntil', 'openrv24m', 'secapprevolutil', 'totalilhighcreditlimit', 'collecti

# test html

In [21]:
import json
import requests
# return prediction with tmp param
keys = ["int_rate"
, "loan_amnt"
, "term"
, "fund_rate"
, "bc_open_to_buy"
, "total_il_high_credit_limit"
, "dti"
, "annual_inc"
, "bc_util"]
data = [14.34, 15000, 36, 25.0, 98.0, 2.8, 1.31, 0.53, 10]
send_data = dict(zip(keys, data))

j_data = json.dumps(send_data)
# print(j_data)
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
url = 'http://127.0.0.1:5000/predict'
r2 = requests.post(url, data=j_data, headers=headers)

In [22]:
r2

<Response [200]>

In [24]:
r2.text

'{\n  "high_risk": 0.09, \n  "safe": 0.91, \n  "threshold": 0.1\n}\n'

In [15]:
import json
prediction = r2.text
json.loads(r2.text)

{'high_risk': 0.09, 'safe': 0.91, 'threshold': 0.1}

In [16]:
a = {"a":1}

In [17]:
a["a"]

1

In [20]:
a.get("B")==None

True

In [26]:
r2 = requests.get('http://localhost:5000/threshold')

In [29]:
r2.json()

0.1

In [30]:
r2.json()

0.1

In [ ]:
r2

In [ ]:
Flask.request()

In [39]:
data = { 'a':'1' , 'b':'2' , 'c':'3' }
data_new = {}
for a,x in data.items():
    data_new.update({a:float(x)})

print(data_new)

a 1.0
b 2.0
c 3.0
{'a': 1.0, 'b': 2.0, 'c': 3.0}
